In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

df_profile = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\Data Processing\\user_Profiles.csv')
df_activity = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\DJANGO\\minor\\website\\csvfile\\user_activity.csv')
df_profile.head(3)

,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_1,veg,vitamin_c,scurvy,high_fiber_diet
1,User_2,non-veg,iron,goitre kidney_disease obesity rickets anemia ...,vegan_diet alkaline_diet ketogenic_diet high_...
2,User_3,veg,vitamin_c,scurvy goitre obesity hypertension anemia pre...,high_fiber_diet


In [2]:
df_activity.head(3)

,User_Id,Meal_Id,Name,catagory,description,Veg_Non,Review,Nutrient,Disease,Diet,Price
0,User_49,meal_id1,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,5,fiber,obesity diabeties goitre hypertension,high_protien_diet high_fiber_diet ketogenic_d...,495
1,User_39,meal_id2,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,10,fiber,kidney_disease obesity hypertension anemia go...,type_a_diet high_protien_diet high_fiber_diet...,435
2,User_77,meal_id3,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,1,vitamin_a,kidney_disease hypertension goitre heart_dise...,high_protien_diet high_fiber_diet paleo_diet ...,390


In [15]:
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\Data Processing\\user_Profiles.csv')
        self.activity = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\DJANGO\\minor\\website\\csvfile\\user_activity.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(self.df.loc[i])
                
#         df_results = df_result
        
        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input[:33]])
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] #taking acitivies
        results = results[results['User_Id']!=user_id] # selecting those which are not reviewed by user
        results = results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results

In [16]:
profile_A = ['calcium','iron','diabeties','gluten_free_diet']
user_id = 'User_71' # asumming for example

In [17]:
ob = Recommender()
data = ob.get_features()
total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
for i in profile_A:
    d[i] = 1

final_input = list(d.values())
similar_users = ob.k_neighbor([final_input[:33]]) # pass 2d array []
similar_users

c:\Users\SUVARNA\AppData\Local\Programs\Python\Python3.9\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_94,veg,potassium,diabeties,low_fat_diet
1,User_28,veg,calcium,diabeties,high_protien_diet vegan_diet ketogenic_diet l...
2,User_60,veg,calcium,diabeties,high_protien_diet vegan_diet ketogenic_diet l...
3,User_91,veg,vitamin_d,kidney_disease,low_fat_diet
4,User_14,veg,protien,goitre,high_fiber_diet
5,User_1,veg,vitamin_c,scurvy,high_fiber_diet


In [18]:
recommendations = ob.user_based(profile_A,user_id) 
recommendations

c:\Users\SUVARNA\AppData\Local\Programs\Python\Python3.9\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Name,Nutrient,Veg_Non,Price,Review,Diet,Disease,description
0,baked namakpara with roasted almond dip,calcium,veg,530,3,high_protien_diet high_fiber_diet ketogenic_d...,kidney_disease goitre hypertension,"almonds (crushed), tomato, garlic cloves, basi..."
1,christmas chocolate fudge cookies,magnesium,veg,445,9,high_fiber_diet ketogenic_diet high_protien_diet,hypertension heart_disease,"unsalted butter, brown sugar, chocolate, choco..."
2,sous-vide salmon tikka,calcium,non-veg,595,7,ketogenic_diet low_fat_diet vegan_diet high_p...,hypertension obesity anemia goitre heart_dise...,"norwegian salmon, black garlic pickle, butter,..."
3,roasted spring chicken with root veggies,iron,non-veg,470,8,type_a_diet high_protien_diet high_fiber_diet...,kidney_disease obesity hypertension anemia go...,"whole chicken, thyme, garlic, lemon, orange, s..."
4,peri peri chicken satay,chloride,non-veg,230,4,type_a_diet high_protien_diet low_carb_diet l...,anemia kidney_disease goitre rickets pregnancy,"boneless skinless chicken thigh (trimmed), sal..."
5,garlic soya chicken,iron,non-veg,265,1,type_a_diet high_protien_diet high_fiber_diet...,cancer kidney_disease hypertension anemia goi...,chicken thigh/breast (cut crosswise into 1/2-i...
6,chicken in mahalak sauce,iron,non-veg,615,4,type_a_diet high_protien_diet high_fiber_diet...,anemia kidney_disease goitre rickets pregnancy,"नमक, चीनी, सीजनिंग, तेल, कॉर्न फलोर, चिकन लेग,..."
7,braised lamb shanks,iron,non-veg,235,7,high_fiber_diet vegan_diet low_fat_diet,goitre,"लैंग शैंक्स, जैतून का तेल, प्याज, गाजर, सेलेरी..."
8,prawn potato soup,sodium,veg,225,9,high_protien_diet high_fiber_diet gluten_free...,obesity pregnancy,"प्याज, आलू, प्रॉन्स, अंडे, दूध, मक्खन, हरा धनि..."
9,thai fish curry,vitamin_a,non-veg,435,5,high_protien_diet high_fiber_diet ketogenic_d...,cancer kidney_disease obesity hypertension an...,"fish (cubed), thai green curry paste, oil, oni..."
